In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
plt.style.use('mplstyle')

def draw_hexagon(ax, center, L, rotation_deg=0, **kwargs):
    """
    Draw a regular hexagon with side length L.
    The vertices are computed at angles 0, 60, …, 360 shifted by rotation_deg.
    For pointy‑top hexagons, use rotation_deg=30.
    """
    angles = np.deg2rad(np.array([0, 60, 120, 180, 240, 300, 360]) + rotation_deg)
    x = center[0] + L * np.cos(angles)
    y = center[1] + L * np.sin(angles)
    hexagon = patches.Polygon(np.column_stack([x, y]), closed=True, zorder=2, **kwargs)
    ax.add_patch(hexagon)

def draw_circle(ax, center, radius, **kwargs):
    """
    Draw a circle with the given center and radius.
    """
    circle = plt.Circle(center, radius, zorder=3, **kwargs)
    ax.add_patch(circle)

def plot_beehive_with_labels(L):
    """
    Plot a beehive of pointy‑top hexagons (one center, six surrounding)
    with each hexagon inscribed by a circle. Then, add text labels at the centers.
    
    The labels are arranged as follows:
      - Center hexagon: Two separate text calls ("AAAA" on the first line in dark blue,
        and just below it "GLEAM 02H" in blue).
      - Surroundings (clockwise from right):
          i=0: EEEE
          i=1: DDDD
          i=2: CCCC
          i=3: BBBB
          i=4: GGGG
          i=5: FFFF
    All text is rendered using LaTeX math mode with \textrm for roman.
    """
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.set_aspect('equal')
    
    rotation = 30  # for pointy-top orientation
    centers = []   # will store center coordinates
    
    # Draw central hexagon and circle.
    center = (0, 0)
    centers.append(center)
    draw_hexagon(ax, center, L, rotation_deg=rotation, edgecolor='azure', facecolor='lightgray', lw=1)
    draw_circle(ax, center, L, edgecolor='C1', facecolor='none', lw=1, ls='--')
    
    # For a regular hexagon, centers of adjacent hexagons are L√3 apart.
    d = L * np.sqrt(3)
    
    # Mapping for surrounding hexagons (clockwise order starting from right)
    labels_map = {0: "PC~E", 1: "PC~D", 2: "PC~C", 3: "PC~B", 4: "PC~G", 5: "PC~F"}
    
    # Draw the 6 surrounding hexagons and their circles.
    for i in range(6):
        angle = np.deg2rad(60 * i)
        offset = (d * np.cos(angle), d * np.sin(angle))
        new_center = (center[0] + offset[0], center[1] + offset[1])
        centers.append(new_center)
        draw_hexagon(ax, new_center, L, rotation_deg=rotation,
                     edgecolor='azure', facecolor='lightgray', lw=1)
        draw_circle(ax, new_center, L, edgecolor='C1', facecolor='none', lw=1, ls='--')
    
    # Now add text labels.
    # For the central hexagon, draw two text objects with a small vertical offset.
    # Using LaTeX math mode with \textrm to render in roman.
    ax.text(center[0], center[1] + 0.15*L, r"$\textrm{PC~A}$", fontsize=12,
            ha='center', va='center', color='darkblue', zorder=4)
    ax.text(center[0], center[1] - 0.15*L, r"$\textrm{GLEAM\ 02H}$", fontsize=12,
            ha='center', va='center', color='r', zorder=4)
    
    # For the surrounding hexagons, add a single text label each.
    # The surrounding centers are stored in centers[1:].
    for i, center in enumerate(centers[1:]):
        label = r"$\textrm{" + labels_map[i] + "}$"
        ax.text(center[0], center[1], label, fontsize=12, ha='center', va='center',
                color='darkblue', zorder=4)
    
    # Set plot limits with margin.
    margin = L * 1.1
    ax.set_xlim(-d - margin, d + margin)
    ax.set_ylim(-d - margin, d + margin)
    plt.axis('off')
    plt.savefig('observation_strategy.pdf', bbox_inches="tight")
    plt.savefig('observation_strategy.png', bbox_inches="tight", dpi=600, transparent=True)

# Example usage: Adjust L to change hexagon size.
plot_beehive_with_labels(L=1)


In [ ]:
import numpy as np
from astropy.io import fits
from matplotlib.patches import Circle, Ellipse, Rectangle
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.colors import Normalize, ListedColormap
import warnings
from astropy.wcs import FITSFixedWarning
from astropy.coordinates import SkyCoord
import astropy.units as u

def get_psf_overlay(fits_file, data_shape, scale_factor=1.0, ellipse_color='k', frame_color='w', lw=2):
    """
    Extracts PSF parameters from a FITS file and returns a matplotlib Ellipse object 
    with a bounding box placed in the bottom-left corner.

    Parameters:
    - fits_file (str): Path to the FITS file.
    - data_shape (tuple): Shape of the FITS image (height, width) to determine placement.
    - scale_factor (float): Factor to adjust the PSF size for better visibility.
    - ellipse_color (str): Color of the PSF ellipse.
    - frame_color (str): Color of the bounding box.
    - lw (float): Line width of the ellipse and box.

    Returns:
    - tuple: (Ellipse, Rectangle) representing the PSF and its bounding frame.
    """
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        
        # Extract beam parameters
        bmaj = header.get('BMAJ', 0)  # Major axis in degrees
        bmin = header.get('BMIN', 0)  # Minor axis in degrees
        bpa = header.get('BPA', 0)    # Position angle in degrees

        # Convert beam size to pixels using pixel scale
        if 'CDELT1' in header:
            pixel_scale = np.abs(header['CDELT1']) * 3600  # Convert deg to arcsec
        else:
            pixel_scale = 1.0  # Default to 1 arcsec/pixel if missing

        bmaj_pix = (bmaj * 3600) / pixel_scale  # Convert to pixels
        bmin_pix = (bmin * 3600) / pixel_scale  

    # Apply scale factor if needed
    bmaj_pix *= scale_factor
    bmin_pix *= scale_factor

    # Determine image size and position in the bottom-left corner
    img_height, img_width = data_shape
    x_pos = img_width * 0.1  
    y_pos = img_height * 0.1  

    # Create PSF ellipse
    psf_ellipse = Ellipse((x_pos, y_pos), width=bmin_pix, height=bmaj_pix, angle=bpa, 
                           edgecolor=ellipse_color, facecolor='none', lw=lw)

    # Create bounding box around the PSF
    box_size = max(bmaj_pix, bmin_pix) * 10  # Slightly larger than the PSF
    psf_bbox = Rectangle((x_pos - box_size / 2, y_pos - box_size / 2), 
                         width=box_size, height=box_size,
                         edgecolor=frame_color, facecolor='none', lw=lw)

    return psf_ellipse, psf_bbox



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.colors import Normalize
import warnings
from astropy.wcs import FITSFixedWarning
plt.style.use('mplstyle')

# Suppress WCS warnings
warnings.simplefilter('ignore', FITSFixedWarning)

def plot_fits_layout(fits_files, cmap='hot_r', vmin=-1e-1, vmax=30, dpi=150, savefile=None, nan_color='white'):
    fig = plt.figure(figsize=(12, 12), dpi=dpi)  # Increased figure size
    fig.subplots_adjust(right=0.88)  # Adjusted for better colorbar placement

    data_list, wcs_list, positions = [], [], []
    common_shape = None  

    # Load data and extract individual WCS coordinates
    for fits_file in fits_files:
        with fits.open(fits_file) as hdul:
            header = hdul[0].header
            wcs = WCS(header, naxis=2) if 'CTYPE1' in header and 'CTYPE2' in header else None
            data = hdul[0].data
            while data.ndim > 2:
                data = data[0]  
            #data = np.nan_to_num(data, nan=np.nanmin(data), posinf=np.nanmax(data), neginf=np.nanmin(data))
            data *= 1e3  # Scaling factor
            data_list.append(data)
            wcs_list.append(wcs)
            if common_shape is None:
                common_shape = data.shape  
            
            # Read RA/Dec from FITS header (image center coordinates)
            ra = header.get('CRVAL1', 0)  # Default to 0 if missing
            dec = header.get('CRVAL2', 0)
            positions.append((ra, dec))

    # Normalize color scale
    norm = Normalize(vmin=vmin, vmax=vmax)

    # Modify colormap to include a specific color for NaNs
    base_cmap = plt.get_cmap(cmap)
    cmap_colors = base_cmap(np.linspace(0, 1, 256))
    cmap_colors[0] = plt.matplotlib.colors.to_rgba(nan_color)  # Set lowest value to nan_color
    modified_cmap = ListedColormap(cmap_colors)


    
    # Arrange panels according to their actual RA/Dec
    ra_vals, dec_vals = zip(*positions)

    axes = []
    im = None  # Placeholder for colorbar reference

    # Identify rows based on unique Dec values
    unique_dec_vals = sorted(set(dec_vals), reverse=True)
    
    for idx in range(len(data_list)):
        wcs = wcs_list[idx]
        data = data_list[idx]

        ax = fig.add_axes([0.5 - (ra_vals[idx] - np.mean(ra_vals)) * 0.22,  
                            0.5 + (dec_vals[idx] - np.mean(dec_vals)) * 0.22,  
                            0.27, 0.27], projection=wcs if wcs is not None else None)

        #ax.set_xticks([])
        im = ax.imshow(data, origin='lower', cmap=cmap, norm=norm)  # Store for colorbar

        # Apply NaN mask (force background color)
        # im.set_array(np.ma.masked_where(nan_mask, data))
        
        # Add panel label inside at the top-right corner
        field = str(fits_files[idx].split('.')[0][0])
        field_name = fr'$\rm{{PC \ {field}}}$'
        # field_name = r'$\rm{PC}$ '+rf'$\rm{{field}}$ '
        ax.text(0.96, 0.96, field_name, color='white', fontsize=12,
                ha='right', va='top', transform=ax.transAxes,
                bbox=dict(facecolor='black', alpha=0.7, edgecolor='none'))

        # # Share y-axis within rows: Only leftmost panel in each row keeps tick labels
        # row_idx = unique_dec_vals.index(dec_vals[idx])  # Identify row
        # leftmost_idx = dec_vals.index(unique_dec_vals[row_idx])  # First in row
        # if idx != leftmost_idx:
        ax.set_yticks([])
        ax.set_yticklabels([])  # Remove tick labels explicitly
        
        if wcs is not None:
            ax.coords.grid(False, color='white', ls='dotted')
            ax.coords[0].set_axislabel("")  # Empty X-axis label
            ax.coords[1].set_axislabel("")  # Empty Y-axis label

        psf_ellipse, psf_bbox = get_psf_overlay(fits_file, data.shape, scale_factor=1.0, ellipse_color='r', frame_color='k', lw=1)
        # ax.add_patch(psf_bbox)    # Add bounding box
        # ax.add_patch(psf_ellipse) # Add PSF ellipse


        axes.append(ax)

        if idx == 2:  # Set y-label at E (leftmost panel of middle row)
            ax.set_ylabel(r"$\mathrm{Declination\ (Dec)}$", labelpad=1)  
        if idx == 3:  # Set x-label below A
            ax.set_xlabel(r"$\mathrm{Right\ Ascension\ (RA)}$", labelpad=18)  
    
    # Add common colorbar to the rightmost part of the plot
    cbar_ax = fig.add_axes([1.15, 0.17, 0.02, 0.9])  # Right edge, thin width
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.set_label(r"$\mathrm{Flux\ Density\ (Jy/beam)}$")
    

    # Save or show
    plt.tight_layout()
    if savefile:
        plt.savefig(savefile, bbox_inches='tight', dpi=dpi)
    else:
        plt.show()

# Example usage
input_images = ["CCCC.SP2B.PBCOR.FITS", "DDDD.SP2B.PBCOR.FITS", "BBBB.SP2B.PBCOR.FITS", "AAAA.SP2B.PBCOR.FITS", 
                 "EEEE.SP2B.PBCOR.FITS", "GGGG.SP2B.PBCOR.FITS", "FFFF.SP2B.PBCOR.FITS" ]  # 7 images

# plot_fits_layout(input_images, savefile="fits_plot.pdf")
plot_fits_layout(input_images, savefile="fits_plot.pdf", dpi=150)
# plot_fits_layout(input_images)


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from astropy.io import fits
# from astropy.wcs import WCS
# from matplotlib.colors import Normalize, LogNorm
# import warnings
# from astropy.wcs import FITSFixedWarning
# plt.style.use('mplstyle')

# # Suppress WCS warnings
# warnings.simplefilter('ignore', FITSFixedWarning)

# def plot_fits_layout(fits_files, cmap='magma', vmin=-1e-1, vmax=30, dpi=150, savefile=None, nan_color='black'):
#     fig = plt.figure(figsize=(12, 12), dpi=dpi)  # Increased figure size
#     fig.subplots_adjust(right=0.88)  # Adjusted for better colorbar placement

#     data_list, wcs_list, positions = [], [], []
#     common_shape = None  

#     # Load data and extract individual WCS coordinates
#     for fits_file in fits_files:
#         with fits.open(fits_file) as hdul:
#             header = hdul[0].header
#             wcs = WCS(header, naxis=2) if 'CTYPE1' in header and 'CTYPE2' in header else None
#             data = hdul[0].data
#             while data.ndim > 2:
#                 data = data[0]  
#             #data = np.nan_to_num(data, nan=np.nanmin(data), posinf=np.nanmax(data), neginf=np.nanmin(data))
#             data *= 1e3  # Scaling factor
#             data_list.append(data)
#             wcs_list.append(wcs)
#             if common_shape is None:
#                 common_shape = data.shape  
            
#             # Read RA/Dec from FITS header (image center coordinates)
#             ra = header.get('CRVAL1', 0)  # Default to 0 if missing
#             dec = header.get('CRVAL2', 0)
#             positions.append((ra, dec))

#     # Normalize color scale
#     norm = Normalize(vmin=vmin, vmax=vmax)
#     # norm = LogNorm(vmin=vmin, vmax=vmax)

#     # Modify colormap to include a specific color for NaNs
#     base_cmap = plt.get_cmap(cmap)
#     cmap_colors = base_cmap(np.linspace(0, 1, 256))
#     cmap_colors[0] = plt.matplotlib.colors.to_rgba(nan_color)  # Set lowest value to nan_color
#     modified_cmap = ListedColormap(cmap_colors)


    
#     # Arrange panels according to their actual RA/Dec
#     ra_vals, dec_vals = zip(*positions)

#     axes = []
#     im = None  # Placeholder for colorbar reference

#     # Identify rows based on unique Dec values
#     unique_dec_vals = sorted(set(dec_vals), reverse=True)
    
#     for idx in range(len(data_list)):
#         wcs = wcs_list[idx]
#         data = data_list[idx]

#         ax = fig.add_axes([0.5 - (ra_vals[idx] - np.mean(ra_vals)) * 0.22,  
#                             0.5 + (dec_vals[idx] - np.mean(dec_vals)) * 0.22,  
#                             0.27, 0.27], projection=wcs if wcs is not None else None)

#         #ax.set_xticks([])
#         im = ax.imshow(data, origin='lower', cmap=cmap, norm=norm)  # Store for colorbar

#         # Apply NaN mask (force background color)
#         # im.set_array(np.ma.masked_where(nan_mask, data))
        
#         # Add panel label inside at the top-right corner
#         ax.text(0.98, 0.98, fits_files[idx].split('.')[0], color='white', fontsize=12,
#                 ha='right', va='top', transform=ax.transAxes,
#                 bbox=dict(facecolor='black', alpha=0.7, edgecolor='none'))

#         # # Share y-axis within rows: Only leftmost panel in each row keeps tick labels
#         # row_idx = unique_dec_vals.index(dec_vals[idx])  # Identify row
#         # leftmost_idx = dec_vals.index(unique_dec_vals[row_idx])  # First in row
#         # if idx != leftmost_idx:
#         ax.set_yticks([])
#         ax.set_yticklabels([])  # Remove tick labels explicitly
        
#         if wcs is not None:
#             ax.coords.grid(True, color='white', ls='dotted')
#             ax.coords[0].set_axislabel("")  # Empty X-axis label
#             ax.coords[1].set_axislabel("")  # Empty Y-axis label

#         psf_ellipse, psf_bbox = get_psf_overlay(fits_file, data.shape, scale_factor=1.0, ellipse_color='r', frame_color='k', lw=1)
#         ax.add_patch(psf_bbox)    # Add bounding box
#         ax.add_patch(psf_ellipse) # Add PSF ellipse


#         axes.append(ax)

#         if idx == 2:  # Set y-label at E (leftmost panel of middle row)
#             ax.set_ylabel(r"$\mathrm{Declination\ (Dec)}$", labelpad=1)  
#         if idx == 3:  # Set x-label below A
#             ax.set_xlabel(r"$\mathrm{Right\ Ascension\ (RA)}$", labelpad=18)  
    
#     # Add common colorbar to the rightmost part of the plot
#     cbar_ax = fig.add_axes([1.15, 0.17, 0.02, 0.9])  # Right edge, thin width
#     cbar = fig.colorbar(im, cax=cbar_ax)
#     cbar.set_label(r"$\mathrm{Flux\ Density\ (Jy/beam)}$")
    

#     # Save or show
#     plt.tight_layout()
#     if savefile:
#         plt.savefig(savefile, bbox_inches='tight', dpi=dpi)
#     else:
#         plt.show()

# # Example usage
# input_images = ["CCCC.SP2B.PBCOR.FITS", "DDDD.SP2B.PBCOR.FITS", "BBBB.SP2B.PBCOR.FITS", "AAAA.SP2B.PBCOR.FITS", 
#                  "EEEE.SP2B.PBCOR.FITS", "GGGG.SP2B.PBCOR.FITS", "FFFF.SP2B.PBCOR.FITS" ]  # 7 images

# plot_fits_layout(input_images)
# # plot_fits_layout(input_images, savefile="fits_plot.pdf")
# # plot_fits_layout(input_images, savefile="fits_plot.png", dpi=300)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.colors import Normalize, ListedColormap
import warnings
from astropy.wcs import FITSFixedWarning
from matplotlib.patches import Ellipse

plt.style.use('mplstyle')

warnings.simplefilter('ignore', FITSFixedWarning)

def plot_mosaic_fits(fits_file, cmap='magma', vmin=-1e-1, vmax=30, dpi=150, savefile=None, nan_color='black'):
    fig = plt.figure(figsize=(12, 12), dpi=dpi)
    
    # Load FITS data
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header, naxis=2) if 'CTYPE1' in header and 'CTYPE2' in header else None
        data = hdul[0].data
        while data.ndim > 2:
            data = data[0]  
        data *= 1e3
    norm = Normalize(vmin=vmin, vmax=vmax)

    # Modify colormap to include NaN handling
    base_cmap = plt.get_cmap(cmap)
    cmap_colors = base_cmap(np.linspace(0, 1, 256))
    cmap_colors[0] = plt.matplotlib.colors.to_rgba(nan_color)
    modified_cmap = ListedColormap(cmap_colors)

    # Main image plot
    ax = fig.add_subplot(111, projection=wcs) if wcs is not None else fig.add_subplot(111)
    im = ax.imshow(data, origin='lower', cmap=modified_cmap, norm=norm)

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, pad=0.01, fraction=0.02, aspect=48.5)
    cbar.set_label(r"$\mathrm{Flux\ Density\ (mJy/beam)}$")
    
    ax.set_xticks([])
    ax.set_yticks([])

    if wcs is not None:
        ax.coords.grid(True, color='white', ls='dotted')
        ax.coords[0].set_axislabel(r"$\mathrm{Right\ Ascension\ (RA)}$")
        ax.coords[1].set_axislabel(r"$\mathrm{Declination\ (Dec)}$")

        # psf_ellipse = get_psf_ellipse(fits_file, data.shape, scale_factor=1.0, color='k', lw=2)
        # ax.add_patch(psf_ellipse)
        psf_ellipse, psf_bbox = get_psf_overlay(fits_file, data.shape, scale_factor=1.0, ellipse_color='r', frame_color='k', lw=1)
        # ax.add_patch(psf_bbox)    # Add bounding box
        # ax.add_patch(psf_ellipse) # Add PSF ellipse

    plt.tight_layout()
    if savefile:
        plt.savefig(savefile, bbox_inches='tight', dpi=dpi)
    else:
        plt.show()

# Example usage
mosaic_fits_file = "final_mosaic.fits"
# plot_mosaic_fits(mosaic_fits_file)
# plot_mosaic_fits(mosaic_fits_file, savefile="mosaic_plot_with_psf.png")
# plot_mosaic_fits(mosaic_fits_file, savefile="mosaic_plot_with_psf.pdf")
plot_mosaic_fits(mosaic_fits_file, savefile="mosaic_plot_with_psf.pdf", dpi=150)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.colors import Normalize
import warnings
from astropy.wcs import FITSFixedWarning
from matplotlib.patches import Rectangle

warnings.simplefilter('ignore', FITSFixedWarning)

def plot_mosaic_fits_triple(fits_file, fovs=(3, 1.5, 0.5), cmap='gist_yarg', vmin=-1e-1, vmax=30, dpi=300, savefile=None):
    fig, axes = plt.subplots(1, 3, figsize=(16, 7), dpi=dpi)

    # Load data and extract WCS
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header, naxis=2) if 'CTYPE1' in header and 'CTYPE2' in header else None
        data = hdul[0].data
        while data.ndim > 2:
            data = data[0]  
        data *= 1e3  # Convert to mJy/beam
    
    # Extract pixel scale
    cdelt1, cdelt2 = abs(header['CDELT1']), abs(header['CDELT2'])  # Degrees per pixel
    ny, nx = data.shape  # Image dimensions
    x_cen, y_cen = nx // 2, ny // 2

    # Normalize color scale
    norm = Normalize(vmin=vmin, vmax=vmax)
    
    ax_list = []
    crop_regions = []  # Store crop regions for each panel
    for i, fov in enumerate(fovs):
        fov_pixels_x = int(fov / cdelt1)  # Pixels corresponding to FOV degrees
        fov_pixels_y = int(fov / cdelt2)
        
        # Define cropping region
        x_min, x_max = x_cen - fov_pixels_x // 2, x_cen + fov_pixels_x // 2
        y_min, y_max = y_cen - fov_pixels_y // 2, y_cen + fov_pixels_y // 2
        
        cropped_data = data[y_min:y_max, x_min:x_max]
        crop_regions.append((x_min, x_max, y_min, y_max))  # Store crop boundaries

        # Update WCS to reflect the cropped region
        if wcs is not None:
            cropped_wcs = wcs.deepcopy()
            cropped_wcs.wcs.crpix[0] -= x_min
            cropped_wcs.wcs.crpix[1] -= y_min
            axes[i].remove()
            ax = fig.add_subplot(1, 3, i + 1, projection=cropped_wcs)
        else:
            ax = axes[i]

        ax_list.append(ax)

        im = ax.imshow(cropped_data, origin='lower', cmap=cmap, norm=norm)

        
        # Apply WCS grid and labels
        if wcs is not None:
            ax.coords.grid(False, color='white', ls='dotted')
            if i == 0:
                ax.coords[1].set_axislabel(r"$\mathrm{Declination\ (Dec)}$")
            else:
                ax.coords[0].set_axislabel("")
                ax.coords[1].set_axislabel("")
            ax.coords[0].set_axislabel(r"$\mathrm{Right\ Ascension\ (RA)}$")

            ax.coords[0].set_ticklabel(visible=True)
            ax.coords[1].set_ticklabel(visible=True)
            ax.coords[0].set_ticks_visible(True)
            ax.coords[1].set_ticks_visible(True)

            # # Major and minor ticks
            # ax.coords[0].set_ticks(number=5, size=5)
            # # ax.coords[0].set_ticks(number=10, size=2,)
            # ax.coords[1].set_ticks(number=5, size=5)
            # # ax.coords[1].set_ticks(number=10, size=2)

        else:
            ax.set_xticks([]) 
            ax.set_yticks([])

    # Extract crop regions
    (x_min_3, x_max_3, y_min_3, y_max_3) = crop_regions[0]  # 3° FoV
    (x_min_1_5, x_max_1_5, y_min_1_5, y_max_1_5) = crop_regions[1]  # 1.5° FoV

    # Compute rectangle positions relative to cropped images
    large_fov_size = int(fovs[0] / cdelt1)  # 3° FoV size in pixels
    medium_fov_size = int(fovs[1] / cdelt1)  # 1.5° FoV size in pixels
    small_fov_size = int(fovs[2] / cdelt1)  # 0.5° FoV size in pixels

    # Rectangle for 1.5° zoom-in (in 3° panel)
    rect1_x_min = (x_min_1_5 - x_min_3)
    rect1_y_min = (y_min_1_5 - y_min_3)

    rect1 = Rectangle((rect1_x_min, rect1_y_min), medium_fov_size, medium_fov_size, 
                      edgecolor='cyan', facecolor='none', lw=2, ls='--', zorder=12)
    ax_list[0].add_patch(rect1)

    # Rectangle for 0.5° zoom-in (in 1.5° panel)
    rect2_x_min = (x_cen - x_min_1_5) - (small_fov_size // 2)
    rect2_y_min = (y_cen - y_min_1_5) - (small_fov_size // 2)

    rect2 = Rectangle((rect2_x_min, rect2_y_min), small_fov_size, small_fov_size, 
                      edgecolor='#E3256B', facecolor='none', lw=2, ls='--', zorder=12)
    ax_list[1].add_patch(rect2)

    # Add colorbar
    # cbar_ax = fig.add_axes([0.91, 0.175, 0.015, 0.728]) 
    cbar_ax = fig.add_axes([0.95, 0.27, 0.015, 0.545]) 
    cbar = fig.colorbar(im, cax=cbar_ax)    
    cbar.set_label(r"$\mathrm{Flux\ Density\ (mJy/beam)}$")
    
    # Reduce spacing between panels
    plt.subplots_adjust(wspace=-0.38)  # Adjust horizontal gap
    plt.tight_layout(rect=[0, 0, 0.95, 1])

    if savefile:
        plt.savefig(savefile, bbox_inches='tight', dpi=dpi)
    plt.show()

# Example usage
plot_mosaic_fits_triple(mosaic_fits_file, fovs=(3, 1.5, 0.5), savefile="mosaic_plot_triple.pdf", dpi=150)
# plot_mosaic_fits_triple(mosaic_fits_file, fovs=(3, 1.5, 0.5), savefile="mosaic_plot_triple.png", dpi=300)


In [ ]:
from astropy.coordinates import SkyCoord

# List of FITS files
fits_files = ["CCCC.SP2B.PBCOR.FITS", "DDDD.SP2B.PBCOR.FITS", "BBBB.SP2B.PBCOR.FITS", "AAAA.SP2B.PBCOR.FITS", 
              "EEEE.SP2B.PBCOR.FITS", "GGGG.SP2B.PBCOR.FITS", "FFFF.SP2B.PBCOR.FITS"]

# Initialize an empty list to store RA and Dec values
ra_dec_list = []

# Extract RA and Dec from the headers and save them in the list
for fits_file in fits_files:
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        ra = header.get('CRVAL1', 'Not found')  # RA value
        dec = header.get('CRVAL2', 'Not found')  # Dec value
        ra_dec_list.append((ra, dec))  # Append tuple of (RA, Dec)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.patches import Circle
from matplotlib.colors import Normalize, ListedColormap
import warnings
from astropy.wcs import FITSFixedWarning
from astropy.coordinates import SkyCoord
import astropy.units as u

plt.style.use('mplstyle')
warnings.simplefilter('ignore', FITSFixedWarning)

def plot_mosaic_fits_with_circles(fits_file, circle_coords, radius_deg=0.2, cmap='Grays', vmin=-1e-2, vmax=10, dpi=150, savefile=None, nan_color='black'):
    fig = plt.figure(figsize=(12, 12), dpi=dpi)
    
    # Load data and extract WCS
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header, naxis=2) if 'CTYPE1' in header and 'CTYPE2' in header else None
        data = hdul[0].data
        while data.ndim > 2:
            data = data[0]  
        
        # Replace NaNs with the lowest valid value
        nan_mask = np.isnan(data)
        # data = np.nan_to_num(data, nan=np.nanmin(data), posinf=np.nanmax(data), neginf=np.nanmin(data))
        data *= 1e3  # Convert to mJy

    # Normalize colormap
    norm = Normalize(vmin=vmin, vmax=vmax)

    # Modify colormap to include a specific color for NaNs
    base_cmap = plt.get_cmap(cmap)
    cmap_colors = base_cmap(np.linspace(0, 1, 256))
    cmap_colors[0] = plt.matplotlib.colors.to_rgba(nan_color)  # Set lowest value to nan_color
    modified_cmap = ListedColormap(cmap_colors)

    # Create subplot with WCS projection if available
    ax = fig.add_subplot(111, projection=wcs) if wcs is not None else fig.add_subplot(111)
    im = ax.imshow(data, origin='lower', cmap=modified_cmap, norm=norm)

    # Apply NaN mask (force background color)
    im.set_array(np.ma.masked_where(nan_mask, data))

    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, pad=0.01, fraction=0.02, aspect=48.5)
    cbar.set_label(r"$\mathrm{Flux\ Density\ (mJy/beam)}$")
    
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Draw circles and field names
    if wcs is not None:
        for name, ra, dec in circle_coords:
            sky_coord = SkyCoord(ra, dec, frame='icrs')
            x, y = wcs.world_to_pixel(sky_coord)

            # Draw the circle
            circle = Circle((x, y), radius_deg / abs(wcs.wcs.cdelt[0]), edgecolor='orange', ls='--', alpha=0.5, facecolor='none', lw=1.5)
            ax.add_patch(circle)

            # Add field name text
            
            # # Add panel label inside at the top-right corner
            # field = str(fits_files[idx].split('.')[0][0])
            # field_name = fr'$\rm{{PC \ {field}}}$'
            # # field_name = r'$\rm{PC}$ '+rf'$\rm{{field}}$ '
            # ax.text(0.96, 0.96, field_name, color='white', fontsize=12,
            #         ha='right', va='top', transform=ax.transAxes,
            #         bbox=dict(facecolor='black', alpha=0.7, edgecolor='none'))

            
            ax.text(x, y + 20, rf"$\mathbf{{{name}}}$", color='white', fontsize=10, ha='center', va='bottom', 
                    bbox=dict(facecolor='C4', alpha=0.8, edgecolor='none'))

        # Apply WCS-specific grid and labels
        ax.coords.grid(False, color='white', ls='dotted')
        ax.coords[0].set_axislabel(r"$\mathrm{Right\ Ascension\ (RA)}$")
        ax.coords[1].set_axislabel(r"$\mathrm{Declination\ (Dec)}$")

    # Save or show
    plt.tight_layout()
    if savefile:
        plt.savefig(savefile, bbox_inches='tight', dpi=dpi)
    else:
        plt.show()

# Example usage:
rms_fits_file = "final_mosaic.pybdsf_rms.fits"
circle_coords = [
    ("PC~A", "02h00m12.00s", "-30d53m23.99s"),
    ("PC~B", "02h06m31.11s", "-30d53m23.99s"),
    ("PC~C", "02h03m21.56s", "-29d31m19.19s"),
    ("PC~D", "01h57m02.44s", "-29d31m19.19s"),
    ("PC~E", "01h53m52.89s", "-30d53m23.98s"),
    ("PC~F", "01h57m02.44s", "-32d15m28.79s"),
    ("PC~G", "02h03m21.56s", "-32d15m28.79s")
]

# plot_mosaic_fits_with_circles(rms_fits_file, circle_coords, radius_deg=2, savefile="rms_mosaic_with_circles.pdf", nan_color='black')


In [ ]:
with fits.open(rms_fits_file) as hdul:
    header = hdul[0].header
    wcs = WCS(header, naxis=2) if 'CTYPE1' in header and 'CTYPE2' in header else None
    data = hdul[0].data
    while data.ndim > 2:
        data = data[0]  
    
    # Replace NaNs with the lowest valid value
    nan_mask = np.isnan(data)
    data = np.nan_to_num(data, nan=np.nanmin(data), posinf=np.nanmax(data), neginf=np.nanmin(data))
    data *= 1e3  # Convert to mJy

    print(data.min(), data.max())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

def plot_cdf(fits_file):
    """
    Plots the cumulative distribution function (CDF) of the RMS map.

    - Left y-axis: Image area as a function of RMS.
    - Right y-axis: Completeness (percentage of area below a given RMS).

    The plot is saved as 'completeness.pdf' and 'completeness.png'.

    Parameters:
    - fits_file: Path to the RMS FITS file.
    """
    # Load RMS map
    with fits.open(fits_file) as hdul_rms:
        rms_data = hdul_rms[0].data[0, :, :]
        hdr = hdul_rms[0].header

    # Compute pixel area in square degrees
    CDELT1 = abs(hdr.get("CDELT1", 1))
    CDELT2 = abs(hdr.get("CDELT2", CDELT1))
    pixel_area_deg2 = CDELT1 * CDELT2  

    # Mask NaN values
    valid_mask = np.isfinite(rms_data)
    total_valid_area = np.sum(valid_mask) * pixel_area_deg2  

    # Flatten RMS values in valid regions (convert to mJy)
    rms_flat = rms_data[valid_mask].flatten() * 1e3  

    # Compute completeness and image area as a function of RMS
    bins = np.logspace(np.log10(np.min(rms_flat)), np.log10(np.max(rms_flat)), 50)
    effective_area = np.array([np.sum(rms_flat <= b) * pixel_area_deg2 for b in bins])
    cdf_percent = 100 * effective_area / total_valid_area  

    # Create figure and twin axes
    fig, ax_cdf = plt.subplots(figsize=(6, 6), dpi=300)
    ax_cdf_twin = ax_cdf.twinx()

    # Plot Image Area vs. RMS
    ax_cdf.plot(bins, effective_area, color="C0", lw=2)
    ax_cdf.set_xlabel(r"$\rm Local\ rms\ (mJy)$")
    ax_cdf.set_ylabel(r"$\rm Image\ Area\ (deg^2)$")
    ax_cdf.tick_params(axis='y')
    ax_cdf.set_xscale('log')
    # ax_cdf.grid(True, which="major", linestyle="--", alpha=0.5)

    # Plot Completeness (CDF)
    ax_cdf_twin.plot(bins, cdf_percent, lw=2, color="C0")
    ax_cdf_twin.set_ylabel(r"$\rm Completeness\ (\%)$", color="k")
    ax_cdf_twin.set_yticks([0, 25, 50, 75, 100])
    ax_cdf_twin.tick_params(axis='y')
    ax_cdf_twin.grid(True, which="major", linestyle="--", alpha=0.5)
    
    # Compute key RMS thresholds for completeness levels
    rms_25 = bins[np.searchsorted(cdf_percent, 25, side="left")]
    rms_50 = bins[np.searchsorted(cdf_percent, 50, side="left")]
    rms_75 = bins[np.searchsorted(cdf_percent, 75, side="left")]
    rms_99 = bins[np.searchsorted(cdf_percent, 98, side="left")]
    rms_90 = bins[np.searchsorted(cdf_percent, 90, side="left")]

    # Plot vertical lines marking completeness thresholds
    # ax_cdf_twin.axvline(x=rms_25, color='C0', linestyle='dashed', linewidth=1, label=r"$\rm 25\%$")
    ax_cdf_twin.axvline(x=rms_50, color='C2', linestyle='dashed', linewidth=1, label=r"$\rm 50\%$")
    # ax_cdf_twin.axvline(x=rms_75, color='C2', linestyle='dashed', linewidth=1, label=r"$\rm 75\%$")
    # ax_cdf_twin.axvline(x=rms_99, color='C2', linestyle='dashed', linewidth=1, label=r"$\rm 98\%$")
    ax_cdf_twin.axvline(x=rms_90, color='C3', linestyle='dashed', linewidth=1, label=r"$\rm 90\%$")

    ax_cdf_twin.legend()

        
    # Save and show the plot
    plt.savefig("completeness.pdf", bbox_inches="tight", dpi=300)

    print(rms_50, rms_90)
    print(rms_50*5, rms_90*5)
    
plot_cdf(rms_fits_file)


In [ ]:
5.44

In [ ]:
from graphviz import Digraph
import IPython.display as display

# Create a Digraph
dot = Digraph(format="pdf")

# Set graph properties
dot.attr(rankdir="TB", nodesep="0.7", ranksep="0.5")  # Adjust spacing
dot.attr("node", shape="rectangle", style="filled", fillcolor="#FFF9DB", fontsize="14")  # Default node attributes

# Define nodes with detailed descriptions in smaller font
dot.node("Start", '''<<TABLE BORDER="0" CELLBORDER="0">
  <TR><TD><B>Observation</B></TD></TR>
  <TR><TD><FONT POINT-SIZE="11">uGMRT Band 2 (GSB)</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Field: GLEAM 02H</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">RA, Dec: Table 1</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Observation strategy: Figure 1</FONT></TD></TR>
</TABLE>>''')

dot.node("Step1", '''<<TABLE BORDER="0" CELLBORDER="0">
  <TR><TD><B>Calibration</B></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Direction-dependent; using SPAM </FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Corrected for DD gains &amp; ionospheric effects</FONT></TD></TR>
</TABLE>>''')

dot.node("Step2", '''<<TABLE BORDER="0" CELLBORDER="0">
  <TR><TD><B>Imaging</B></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Each fields are imaged using AIPS (included in SPAM pipeline)</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Mosaic: PB corrected images are optimally combined using Astropy</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Off-source rms is ~5 mJy</FONT></TD></TR>
</TABLE>>''')

dot.node("Step3_4", '''<<TABLE BORDER="0" CELLBORDER="0">
  <TR><TD><B>Source Finding &amp; Catalog Making</B></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Using PyBDSF</FONT></TD></TR>
</TABLE>>''')

dot.node("Step5", '''<<TABLE BORDER="0" CELLBORDER="0">
  <TR><TD><B>Cross-matching with Catalogs</B></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">GLEAM-X, TGSS, NVSS</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Good positional and flux accuracy with GLEAM-X and TGSS 150 MHz catalog</FONT></TD></TR>
</TABLE>>''')

dot.node("Step6", '''<<TABLE BORDER="0" CELLBORDER="0">
  <TR><TD><B>Spectral Index Distribution</B></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Using 1.4 GHz NVSS catalog</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">&#945;<SUB>mean</SUB> = -0.72</FONT></TD></TR>
</TABLE>>''')

dot.node("Step7", '''<<TABLE BORDER="0" CELLBORDER="0">
  <TR><TD><B>Source Counts</B></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Compute Euclidean-normalized source counts</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Corrections: FDR, Completeness (includes Eddington bias and resolution bias)</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Matches with GLEAM-X source counts and Franzen's model</FONT></TD></TR>
</TABLE>>''')

dot.node("End", '''<<TABLE BORDER="0" CELLBORDER="0">
  <TR><TD><B>Future plan</B></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Deeper Image with GWB</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">In-band spectral index measurement</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Deeper source counts</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Characterisation of diffused Galactic emission</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Deeper observation of GLEAM 2H</FONT></TD></TR>
  <TR><TD><FONT POINT-SIZE="10">Other calibration fields: GLEAM 0H and 5H</FONT></TD></TR>
</TABLE>>''')

# Define edges
dot.edge("Start", "Step1")
dot.edge("Step1", "Step2")
dot.edge("Step2", "Step3_4")  # Merged step
dot.edge("Step3_4", "Step5")  # Merged step
dot.edge("Step5", "Step6")
dot.edge("Step6", "Step7")
dot.edge("Step7", "End")

# Render and save as PDF
pdf_path = dot.render("flowchart")  # Saves as flowchart.pdf

# Display PDF in Jupyter
display.display(display.IFrame(pdf_path, width=600, height=800))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.colors import Normalize, ListedColormap
import warnings
from astropy.wcs import FITSFixedWarning
from matplotlib.patches import Ellipse

plt.style.use('mplstyle')

warnings.simplefilter('ignore', FITSFixedWarning)

def plot_mosaic_fits_proposal(fits_file, cmap='magma', vmin=-1e-2, vmax=30, dpi=150, savefile=None, nan_color='black'):
    fig = plt.figure(figsize=(6, 6), dpi=dpi)
    
    # Load FITS data
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header, naxis=2) if 'CTYPE1' in header and 'CTYPE2' in header else None
        data = hdul[0].data
        while data.ndim > 2:
            data = data[0]  
        data *= 1e3
    norm = Normalize(vmin=vmin, vmax=vmax)

    # Modify colormap to include NaN handling
    base_cmap = plt.get_cmap(cmap)
    cmap_colors = base_cmap(np.linspace(0, 1, 256))
    cmap_colors[0] = plt.matplotlib.colors.to_rgba(nan_color)
    modified_cmap = ListedColormap(cmap_colors)

    # Main image plot
    ax = fig.add_subplot(111, projection=wcs) if wcs is not None else fig.add_subplot(111)
    im = ax.imshow(data, origin='lower', cmap=modified_cmap, norm=norm)

    name, ra, dec = circle_coords[0]
    sky_coord = SkyCoord(ra, dec, frame='icrs')
    x, y = wcs.world_to_pixel(sky_coord)

    # Draw the circle
    radius_deg=3.
    circle = Circle((x, y), radius_deg / abs(wcs.wcs.cdelt[0]), edgecolor='white', ls='--', alpha=0.8, facecolor='none', lw=1.5)
    ax.add_patch(circle)

    # # Draw the circle
    # radius_deg=10
    # circle = Circle((x, y), radius_deg / abs(wcs.wcs.cdelt[0]), edgecolor='k', ls='--', alpha=0.5, facecolor='none', lw=1.5)
    # ax.add_patch(circle)

    # Compute radius in pixels
    radius_pix = radius_deg / abs(wcs.wcs.cdelt[0])
    
    # Adjust axis limits to make sure full 4° circle is visible
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # Compute new limits centered on (x, y), extended to fit the circle
    x_min = min(xlim[0], x - radius_pix * 1.05)
    x_max = max(xlim[1], x + radius_pix * 1.05)
    y_min = min(ylim[0], y - radius_pix * 1.05)
    y_max = max(ylim[1], y + radius_pix * 1.05)
    
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)


    # Add colorbar
    cbar = plt.colorbar(im, ax=ax, pad=0.01, fraction=0.02, aspect=48.5)
    cbar.set_label(r"$\mathrm{Flux\ Density\ (mJy/beam)}$")
    
    ax.set_xticks([])
    ax.set_yticks([])

    if wcs is not None:
        ax.coords.grid(False, color='white', ls='dotted')
        ax.coords[0].set_axislabel(r"$\mathrm{Right\ Ascension\ (RA)}$")
        ax.coords[1].set_axislabel(r"$\mathrm{Declination\ (Dec)}$")

        # psf_ellipse = get_psf_ellipse(fits_file, data.shape, scale_factor=1.0, color='k', lw=2)
        # ax.add_patch(psf_ellipse)
        psf_ellipse, psf_bbox = get_psf_overlay(fits_file, data.shape, scale_factor=1.0, ellipse_color='r', frame_color='k', lw=1)
        # ax.add_patch(psf_bbox)    # Add bounding box
        # ax.add_patch(psf_ellipse) # Add PSF ellipse

    plt.tight_layout()
    if savefile:
        plt.savefig(savefile, bbox_inches='tight', dpi=dpi)
    else:
        plt.show()

# Example usage
mosaic_fits_file = "final_mosaic.fits"
plot_mosaic_fits_proposal(mosaic_fits_file)
plot_mosaic_fits_proposal(mosaic_fits_file, savefile="mosaic_plot_proposal.pdf")
# plot_mosaic_fits(mosaic_fits_file, savefile="mosaic_plot_with_psf.pdf")
# plot_mosaic_fits(mosaic_fits_file, savefile="mosaic_plot_with_psf.png", dpi=300)


In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib.patches import Circle
# import numpy as np

# def plot_hex_flower_two_layers(radius=3, circle_color='orange', edge_color='black'):
#     fig, ax = plt.subplots(figsize=(10, 10))
    
#     cx, cy = 0.0, 0.0

#     # Enclosing circle (drawn first, at back)
#     enclosing_radius = 10 
#     ax.add_patch(Circle((cx, cy), radius=enclosing_radius,
#                         edgecolor='darkred', facecolor='none', lw=2, ls='--', alpha=0.3, zorder=3))


#     # Enclosing circle (drawn first, at back)
#     enclosing_radius1 =  2  * radius
#     print(enclosing_radius1)
#     ax.add_patch(Circle((cx, cy), radius=enclosing_radius1,
#                         edgecolor='blue', facecolor='none', lw=2, ls='--', alpha=0.8, zorder=3))
    
    
#     # Enclosing circle (drawn first, at back)
#     enclosing_radius0 = 3 
#     ax.add_patch(Circle((cx, cy), radius=enclosing_radius0,
#                         edgecolor='k', facecolor='k', lw=2, ls='-', alpha=0.9, zorder=4))

    
#     # Second layer (outermost 12): 6 + 6
#     r2 = 3. * radius
#     for i in range(6):
#         angle = np.radians(60 * i)
#         x = cx + r2 * np.cos(angle)
#         y = cy + r2 * np.sin(angle)
#         ax.add_patch(Circle((x, y), radius=radius, edgecolor='darkred',
#                             facecolor='darkred', lw=1, alpha=0.3, zorder=1))

    
    
    
#     r3 = 4. * np.sqrt(3)/2 * radius
#     for i in range(6):
#         angle = np.radians(60 * i + 30)
#         x = cx + r3 * np.cos(angle)
#         y = cy + r3 * np.sin(angle)
#         ax.add_patch(Circle((x, y), radius=radius, edgecolor='darkred',
#                             facecolor='darkred', lw=1, alpha=0.3, zorder=1))

#     # First layer: 6 inner surrounding circles
#     r1 = np.sqrt(3) * radius
#     for i in range(6):
#         angle = np.radians(60 * i + 30)
#         x = cx + r1 * np.cos(angle)
#         y = cy + r1 * np.sin(angle)
#         ax.add_patch(Circle((x, y), radius=radius, edgecolor='k',
#                             facecolor=circle_color, lw=1, alpha=0.8, zorder=2))

#     # # Central circle (topmost)
#     # ax.add_patch(Circle((cx, cy), radius=radius, edgecolor='k',
#     #                     facecolor='k', lw=2, alpha=0.3, hatch='o', zorder=3))

#     # Plot settings
#     ax.set_aspect('equal')
#     buffer = enclosing_radius + 5
#     ax.set_xlim(cx - buffer, cx + buffer)
#     ax.set_ylim(cy - buffer, cy + buffer)
#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.set_facecolor('white')
#     # plt.title("Hexagonal Packing: Ordered Layers", fontsize=14)



#     # from matplotlib.patches import Patch
    
#     # # Define proxy artists for legend
#     # legend_elements = [
#     #     Patch(facecolor='k', edgecolor='k', label='Central ' + r'$3^\circ \times 3^\circ$', alpha=0.8, lw=1),
#     #     Patch(facecolor=circle_color, edgecolor=circle_color, label='Inner  ' + r'$6^\circ \times 6^\circ$', alpha=0.8, lw=1),
#     #     Patch(facecolor='darkred', edgecolor='darkred', label='Outer ' + r'$10^\circ \times 10^\circ$', alpha=0.3, lw=1),
#     #     # Patch(facecolor='none', edgecolor='blue', linestyle='--', label=r'$6^\circ$', lw=2),
#     #     # Patch(facecolor='none', edgecolor='red', linestyle='--', label=r'$10^\circ$', lw=2)
#     # ]
    
#     # # Add legend to plot
#     # ax.legend(handles=legend_elements, loc='lower left', fontsize=16, frameon=True)



    
#     # plt.tight_layout()
#     plt.savefig('obs_10deg_ordered.pdf')
#     plt.savefig('obs_10deg_ordered.jpg', dpi=300)
#     plt.show()

# # Run it
# plot_hex_flower_two_layers()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np

def plot_hex_flower_two_layers(radius=3, circle_color='yellow', edge_color='black'):
    fig, ax = plt.subplots(figsize=(10, 10))
    
    cx, cy = 0.0, 0.0

    # Enclosing circle (drawn first, at back)
    enclosing_radius = 10 
    ax.add_patch(Circle((cx, cy), radius=enclosing_radius,
                        edgecolor='darkred', facecolor='none', lw=2, ls='--', alpha=0.9, zorder=3))


    # Enclosing circle (drawn first, at back)
    enclosing_radius1 =  2  * radius
    print(enclosing_radius1)
    ax.add_patch(Circle((cx, cy), radius=enclosing_radius1,
                        edgecolor='blue', facecolor='none', lw=2, ls='--', alpha=0.9, zorder=3))
    
    
    # Enclosing circle (drawn first, at back)
    enclosing_radius0 = 3 
    ax.add_patch(Circle((cx, cy), radius=enclosing_radius0,
                        edgecolor='k', facecolor='k', lw=2, ls='-', alpha=0.9, zorder=4))

    
    # Second layer (outermost 12): 6 + 6
    r2 = 3. * radius
    for i in range(6):
        angle = np.radians(60 * i)
        x = cx + r2 * np.cos(angle)
        y = cy + r2 * np.sin(angle)
        ax.add_patch(Circle((x, y), radius=radius, edgecolor='darkred',
                            facecolor='darkred', lw=1, alpha=0.3, zorder=1))

    
    
    
    r3 = 4. * np.sqrt(3)/2 * radius
    for i in range(6):
        angle = np.radians(60 * i + 30)
        x = cx + r3 * np.cos(angle)
        y = cy + r3 * np.sin(angle)
        ax.add_patch(Circle((x, y), radius=radius, edgecolor='darkred',
                            facecolor='darkred', lw=1, alpha=0.3, zorder=1))

    # First layer: 6 inner surrounding circles
    r1 = np.sqrt(3) * radius
    for i in range(6):
        angle = np.radians(60 * i + 30)
        x = cx + r1 * np.cos(angle)
        y = cy + r1 * np.sin(angle)
        ax.add_patch(Circle((x, y), radius=radius, edgecolor='k',
                            facecolor=circle_color, lw=1, alpha=0.8, zorder=2))

    # # Central circle (topmost)
    # ax.add_patch(Circle((cx, cy), radius=radius, edgecolor='k',
    #                     facecolor='k', lw=2, alpha=0.3, hatch='o', zorder=3))

    # Plot settings
    ax.set_aspect('equal')
    buffer = enclosing_radius + 5
    ax.set_xlim(cx - buffer, cx + buffer)
    ax.set_ylim(cy - buffer, cy + buffer)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_facecolor('white')
    # plt.title("Hexagonal Packing: Ordered Layers", fontsize=14)



    from matplotlib.patches import Patch
    
    # Define proxy artists for legend
    legend_elements = [
        Patch(facecolor='k', edgecolor='k', label=' ' + r'$\rm{{Central}}\,3^\circ$', alpha=0.8, lw=1),
        # Patch(facecolor=circle_color, edgecolor=circle_color, label='Inner  ' + r'$6^\circ \times 6^\circ$', alpha=0.8, lw=1),
        # Patch(facecolor='darkred', edgecolor='darkred', label='Outer ' + r'$10^\circ \times 10^\circ$', alpha=0.3, lw=1),
        Patch(facecolor='none', edgecolor='blue', linestyle='--', label=r'$\rm{{Inner}}\,6^\circ$', lw=2),
        Patch(facecolor='none', edgecolor='red', linestyle='--', label=r'$\rm{{Outer}}\,10^\circ$', lw=2)
    ]
    
    # Add legend to plot
    ax.legend(handles=legend_elements, loc='lower left', fontsize=16, frameon=True)



    
    # plt.tight_layout()
    plt.savefig('obs_10deg_ordered.pdf',  bbox_inches="tight", )
    plt.savefig('obs_10deg_ordered.jpg',  bbox_inches="tight", dpi=300)
    plt.show()

# Run it
plot_hex_flower_two_layers()


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np

def plot_hex_flower_two_layers(radius=3, circle_color='gray', edge_color='black'):
    fig, ax = plt.subplots(figsize=(10, 10))
    
    cx, cy = 0.0, 0.0

    # Enclosing circle (drawn first, at back)
    enclosing_radius = 10 
    ax.add_patch(Circle((cx, cy), radius=enclosing_radius,
                        edgecolor='lime', facecolor='none', lw=3, ls='--', alpha=1, zorder=10))


    # # Enclosing circle (drawn first, at back)
    # enclosing_radius1 =  2  * radius
    # print(enclosing_radius1)
    # ax.add_patch(Circle((cx, cy), radius=enclosing_radius1,
    #                     edgecolor='blue', facecolor='none', lw=2, ls='--', alpha=0.9, zorder=3))
    
    
    # Enclosing circle (drawn first, at back)
    enclosing_radius0 = 3 
    ax.add_patch(Circle((cx, cy), radius=enclosing_radius0,
                        edgecolor='y', facecolor='gray', lw=2, ls='-', alpha=0.8, zorder=1))

    
    # Second layer (outermost 12): 6 + 6
    r2 = 3. * radius
    for i in range(6):
        angle = np.radians(60 * i)
        x = cx + r2 * np.cos(angle)
        y = cy + r2 * np.sin(angle)
        ax.add_patch(Circle((x, y), radius=radius, edgecolor='darkred',
                            facecolor='gray', lw=1, alpha=1, zorder=2))

    
    
    
    r3 = 4. * np.sqrt(3)/2 * radius
    for i in range(6):
        angle = np.radians(60 * i + 30)
        x = cx + r3 * np.cos(angle)
        y = cy + r3 * np.sin(angle)
        ax.add_patch(Circle((x, y), radius=radius, edgecolor='darkred',
                            facecolor='gray', lw=1, alpha=1, zorder=2))

    # First layer: 6 inner surrounding circles
    r1 = np.sqrt(3) * radius
    for i in range(6):
        angle = np.radians(60 * i + 30)
        x = cx + r1 * np.cos(angle)
        y = cy + r1 * np.sin(angle)
        ax.add_patch(Circle((x, y), radius=radius, edgecolor='k',
                            facecolor=circle_color, lw=1, alpha=0.8, zorder=2))

    # Central circle (topmost)
    ax.add_patch(Circle((cx, cy), radius=radius, edgecolor='y',
                        facecolor='gray', lw=1, alpha=1, zorder=1))

    # Plot settings
    ax.set_aspect('equal')
    buffer = enclosing_radius + 5
    ax.set_xlim(cx - buffer, cx + buffer)
    ax.set_ylim(cy - buffer, cy + buffer)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_facecolor('white')
    # plt.title("Hexagonal Packing: Ordered Layers", fontsize=14)



    from matplotlib.patches import Patch
    
    # Define proxy artists for legend
    legend_elements = [
        # Patch(facecolor='k', edgecolor='k', label=' ' + r'$\rm{{Central}}\,3^\circ$', alpha=0.8, lw=1),
        # Patch(facecolor=circle_color, edgecolor=circle_color, label='Inner  ' + r'$6^\circ \times 6^\circ$', alpha=0.8, lw=1),
        # Patch(facecolor='darkred', edgecolor='darkred', label='Outer ' + r'$10^\circ \times 10^\circ$', alpha=0.3, lw=1),
        # Patch(facecolor='none', edgecolor='blue', linestyle='--', label=r'$\rm{{Inner}}\,6^\circ$', lw=2),
        # Patch(facecolor='none', edgecolor='red', linestyle='--', label=r'$\rm{{Outer}}\,10^\circ$', lw=2)
    ]
    
    # Add legend to plot
    # ax.legend(handles=legend_elements, loc='lower left', fontsize=16, frameon=True)



    plt.axis('off')
    # plt.tight_layout()
    # plt.savefig('obs_10deg_ordered.pdf',  bbox_inches="tight", )
    # plt.savefig('obs_10deg_ordered.jpg',  bbox_inches="tight", dpi=300)
    plt.savefig('obs_10deg_ordered.png', bbox_inches="tight", dpi=300, transparent=True)

    plt.show()

# Run it
plot_hex_flower_two_layers()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.colors import Normalize, ListedColormap
import warnings
from astropy.wcs import FITSFixedWarning
from matplotlib.patches import Ellipse

plt.style.use('mplstyle')

warnings.simplefilter('ignore', FITSFixedWarning)

def plot_mosaic_fits(fits_file, cmap='magma_r', vmin=-1e-1, vmax=50, dpi=150, savefile=None, nan_color='white'):
    fig = plt.figure(figsize=(12, 12), dpi=dpi)
    
    # Load FITS data
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header, naxis=2) if 'CTYPE1' in header and 'CTYPE2' in header else None
        data = hdul[0].data
        while data.ndim > 2:
            data = data[0]  
        data *= 1e3
    norm = Normalize(vmin=vmin, vmax=vmax)

    # Modify colormap to include NaN handling
    base_cmap = plt.get_cmap(cmap)
    cmap_colors = base_cmap(np.linspace(0, 1, 256))
    cmap_colors[0] = plt.matplotlib.colors.to_rgba(nan_color)
    modified_cmap = ListedColormap(cmap_colors)

    # Main image plot
    ax = fig.add_subplot(111, projection=wcs) if wcs is not None else fig.add_subplot(111)
    im = ax.imshow(data, origin='lower', cmap=modified_cmap, norm=norm)

    # Add colorbar
    # cbar = plt.colorbar(im, ax=ax, pad=0.01, fraction=0.02, aspect=48.5)
    # cbar.set_label(r"$\mathrm{Flux\ Density\ (mJy/beam)}$")
    
    ax.set_xticks([])
    ax.set_yticks([])

    if wcs is not None:
        ax.coords.grid(True, color='white', ls='dotted')
        ax.coords[0].set_axislabel(r"$\mathrm{Right\ Ascension\ (RA)}$")
        ax.coords[1].set_axislabel(r"$\mathrm{Declination\ (Dec)}$")

        # psf_ellipse = get_psf_ellipse(fits_file, data.shape, scale_factor=1.0, color='k', lw=2)
        # ax.add_patch(psf_ellipse)
        # psf_ellipse, psf_bbox = get_psf_overlay(fits_file, data.shape, scale_factor=1.0, ellipse_color='r', frame_color='k', lw=1)
        # ax.add_patch(psf_bbox)    # Add bounding box
        # ax.add_patch(psf_ellipse) # Add PSF ellipse
    plt.axis('off')
    plt.tight_layout()
    if savefile:
        
        plt.savefig(savefile, bbox_inches="tight", dpi=300, transparent=True)
    else:
        plt.show()

# Example usage
mosaic_fits_file = "final_mosaic.fits"
plot_mosaic_fits(mosaic_fits_file, savefile="mosaic_plot.png", dpi=300)
# plot_mosaic_fits(mosaic_fits_file, )


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from matplotlib.colors import Normalize, ListedColormap
import warnings
from astropy.wcs import FITSFixedWarning
from matplotlib.patches import Ellipse

plt.style.use('mplstyle')

warnings.simplefilter('ignore', FITSFixedWarning)

def plot_mosaic_fits(fits_file, cmap='magma_r', vmin=-1e-1, vmax=50, dpi=150, savefile=None, nan_color='white'):
    fig = plt.figure(figsize=(12, 12), dpi=dpi)
    
    # Load FITS data
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header, naxis=2) if 'CTYPE1' in header and 'CTYPE2' in header else None
        data = hdul[0].data
        while data.ndim > 2:
            data = data[0]  
        data *= 1e3
    norm = Normalize(vmin=vmin, vmax=vmax)

    # Modify colormap to include NaN handling
    base_cmap = plt.get_cmap(cmap)
    cmap_colors = base_cmap(np.linspace(0, 1, 256))
    cmap_colors[0] = plt.matplotlib.colors.to_rgba(nan_color)
    modified_cmap = ListedColormap(cmap_colors)

    # Main image plot
    ax = fig.add_subplot(111, projection=wcs) if wcs is not None else fig.add_subplot(111)
    im = ax.imshow(data, origin='lower', cmap=modified_cmap, norm=norm)

    # Add colorbar
    # cbar = plt.colorbar(im, ax=ax, pad=0.01, fraction=0.02, aspect=48.5)
    # cbar.set_label(r"$\mathrm{Flux\ Density\ (mJy/beam)}$")
    
    ax.set_xticks([])
    ax.set_yticks([])

    if wcs is not None:
        ax.coords.grid(True, color='white', ls='dotted')
        ax.coords[0].set_axislabel(r"$\mathrm{Right\ Ascension\ (RA)}$")
        ax.coords[1].set_axislabel(r"$\mathrm{Declination\ (Dec)}$")

        # psf_ellipse = get_psf_ellipse(fits_file, data.shape, scale_factor=1.0, color='k', lw=2)
        # ax.add_patch(psf_ellipse)
        # psf_ellipse, psf_bbox = get_psf_overlay(fits_file, data.shape, scale_factor=1.0, ellipse_color='r', frame_color='k', lw=1)
        # ax.add_patch(psf_bbox)    # Add bounding box
        # ax.add_patch(psf_ellipse) # Add PSF ellipse
    plt.axis('off')
    plt.tight_layout()
    if savefile:
        
        plt.savefig(savefile, bbox_inches="tight", dpi=300, transparent=True)
    else:
        plt.show()

# Example usage
mosaic_fits_file = "final_mosaic.fits"
plot_mosaic_fits(mosaic_fits_file, savefile="mosaic_plot.png", dpi=300)
# plot_mosaic_fits(mosaic_fits_file, )


In [ ]:
ValueError: 'gg' is not a valid value for name; supported values are 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 
'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Grays', 'Grays_r', 'Greens', 'Greens_r', 
'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2',
'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r',
'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds',
'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 
'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'berlin', 'berlin_r', 
'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r',
'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag', 'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_grey', 
'gist_grey_r', 'gist_heat', 'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern', 'gist_stern_r', 'gist_yarg', 
'gist_yarg_r', 'gist_yerg', 'gist_yerg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r', 'gnuplot_r', 'gray', 'gray_r', 'grey', 'grey_r', 'hot', 'hot_r', 'hsv', 
'hsv_r', 'inferno', 'inferno_r', 'jet', 'jet_r', 'magma', 'magma_r', 'managua', 'managua_r', 'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 
'pink_r', 'plasma', 'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'seismic', 'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r', 'tab10', 
'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r', 'terrain', 'terrain_r', 'turbo', 
'turbo_r', 'twilight', 'twilight_r', 'twilight_shifted', 'twilight_shifted_r', 'vanimo', 'vanimo_r', 'viridis', 'viridis_r', 'winter', 'winter_r'